In [ ]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import *
spark = pyspark.sql.SparkSession.builder.appName("Product_Price_Tracking") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

from delta.tables import *

In [ ]:
deltaTable = DeltaTable.forPath(spark, "hdfs:///delta_lake/products")

In [ ]:
df_history = deltaTable.history()
df_history.show(20, False)

In [ ]:
df_lastOperation = deltaTable.history(1)
df_lastOperation.show()

In [ ]:
df_productsaug22 = spark.read.csv('hdfs:///delta_lake/raw/products_aug22.csv', header=True, inferSchema=True)
df_productsaug22.show()     

In [ ]:
deltaTable = DeltaTable.forPath(spark, "hdfs:///delta_lake/products")

In [ ]:
df_productsaug22.write.format("delta").mode("append").save("hdfs:///delta_lake/products")

In [ ]:
df_productsaug22.write.format("delta").mode("append").option("mergeSchema", "true").save("hdfs:///delta_lake/products")

In [ ]:
spark.table("products").show()

In [ ]:
df_productsaug_partition = spark.read.csv('hdfs:///delta_lake/raw/*.csv', header=True, inferSchema=True)

In [ ]:
df_productsaug_partition.write.format("delta").partitionBy("Date").option("path", "hdfs:///delta_lake/products_p").saveAsTable("products_p")

In [ ]:
deltaTable.vacuum() 